In [1]:
import pandas as pd

In [2]:
res_oct = pd.read_csv('./res/oct.csv')
res_boct = pd.read_csv('./res/boct.csv')
res_mfoct = pd.read_csv('./res/mfoct.csv')
res_sk = pd.read_csv('./res/sk.csv')
res = {'OCT':res_oct, 'binOCT':res_boct, 'flowOCT':res_mfoct, 'CART':res_sk}

In [3]:
def resProcess(res):
    """
    Calculate the average results from random data split
    Select alpha with highest validation accuracy
    """
    if 'alpha' in res.columns:
        res_group = res.groupby(['instance', 'depth', 'alpha']).mean().reset_index()
        res_group = res_group.groupby(['instance', 'depth']) \
                             .apply(lambda group: group[group['val_acc'] == group['val_acc'].max()]) \
                             .drop_duplicates(subset=['instance', 'depth']) \
                             .reset_index(drop=True)
    else:
        res_group = res.groupby(['instance', 'depth']).mean().reset_index()
    return res_group

In [4]:
def resCombine(res, feat):
    df = pd.DataFrame(columns=['instance', 'depth'])
    for r in res:
        df_cur = resProcess(res[r])[['instance', 'depth', feat]]
        df_cur = df_cur.rename(columns={feat:r})
        df = df.merge(df_cur, how='outer', on=['instance', 'depth'])
    return df

In [5]:
resCombine(res, 'test_acc').to_csv('./res/res.csv')